In [6]:
# Step 1: Install Required Libraries
# Install TensorFlow and TensorFlow Addons
!pip install tensorflow==2.15.0 tensorflow-addons==0.23.0

In [7]:
#  Step 2: Import Required Libraries

import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
#  Step 3: Check TensorFlow Version & GPU Availability

print("TensorFlow Version:", tf.__version__)  # Should be 2.15.0

# Check if GPU is available
gpu_devices = tf.config.list_physical_devices('GPU')
print("GPU Available:", gpu_devices)

TensorFlow Version: 2.15.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [9]:
#  Step 4: Optimize GPU Memory Usage (If GPU is Available)

if gpu_devices:
    for gpu in gpu_devices:
        tf.config.experimental.set_memory_growth(gpu, True)
    print("✅ GPU Optimization Complete!")

✅ GPU Optimization Complete!


In [26]:
#  Step 5: Define Dataset Paths

# Kaggle automatically mounts the dataset in /kaggle/input/
data_path = "/kaggle/input/raf-db-dataset"  # Adjust if necessary



In [27]:
# Step 6: Data Augmentation & Preprocessing

IMG_SIZE = 224  # Image resolution (224x224 pixels)
BATCH_SIZE = 32  # Number of images per batch

# Define augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Normalize pixel values to [0,1]
    rotation_range=10,      # Rotate images randomly by 10 degrees
    width_shift_range=0.1,  # Shift images horizontally by 10%
    height_shift_range=0.1, # Shift images vertically by 10%
    horizontal_flip=True    # Flip images horizontally
)

# Only normalization for validation data (no augmentation)
val_datagen = ImageDataGenerator(rescale=1./255)

In [29]:
# Step 7: Load Images from Directories

train_ds = train_datagen.flow_from_directory(
    os.path.join(data_path, "/kaggle/input/raf-db-dataset/DATASET/train"),  # Path to training images
    target_size=(IMG_SIZE, IMG_SIZE),  # Resize images to 224x224
    batch_size=BATCH_SIZE,
    class_mode="categorical"  # Multi-class classification
)

val_ds = val_datagen.flow_from_directory(
    os.path.join(data_path, "/kaggle/input/raf-db-dataset/DATASET/test"),   # Path to validation images
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical"
)

print("✅ Dataset Loaded Successfully!")

Found 12271 images belonging to 7 classes.
Found 3068 images belonging to 7 classes.
✅ Dataset Loaded Successfully!


In [30]:
# Step 8: Define Model (Using EfficientNetB0)

from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Load EfficientNetB0 as the feature extractor (pretrained on ImageNet)
base_model = EfficientNetB0(weights="imagenet", include_top=False)
base_model.trainable = False  # Keep pretrained layers frozen

# Build the custom classification model
inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = base_model(inputs, training=False)  # Pass images through the feature extractor
x = GlobalAveragePooling2D()(x)  # Convert feature maps to a single vector
x = Dense(512, activation="relu")(x)  # Fully connected layer with ReLU
x = Dropout(0.5)(x)  # Dropout to prevent overfitting
x = BatchNormalization()(x)  # Normalize activations
outputs = Dense(7, activation="softmax")(x)  # 7 output classes (RAF-DB emotions)

# Create the final model
model = Model(inputs, outputs)

# Compile the model
model.compile(
    optimizer=Adam(learning_rate=1e-4),  # Adam optimizer with a small learning rate
    loss="categorical_crossentropy",  # Multi-class classification loss
    metrics=["accuracy"]
)

# Display model summary
model.summary()

16705208/16705208 [==============================] - 1s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1280)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 512)               655872    
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                              

In [31]:
# Step 9: Define Checkpoint Callback (Save Every 10 Epochs)

# Define where to save checkpoints
checkpoint_path = "/kaggle/working/rafdb_checkpoints/weights_epoch_{epoch:02d}.h5"
os.makedirs("/kaggle/working/rafdb_checkpoints", exist_ok=True)  # Ensure directory exists

# Create the checkpoint callback
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,  # Save model weights
    save_weights_only=True,    # Save only the weights, not the entire model
    save_freq=10 * (len(train_ds)),  # Save at every 10th epoch
    verbose=1
)

In [32]:
# Step 10: Train the Model

EPOCHS = 50  # Total number of training epochs

history = model.fit(
    train_ds,                # Training dataset
    validation_data=val_ds,  # Validation dataset
    epochs=EPOCHS,           # Number of epochs
    callbacks=[checkpoint_callback],  # Save checkpoints every 10 epochs
    verbose=1  # Show detailed progress
)

print("✅ Training Completed!")

Epoch 1/50
384/384 [==============================] - 242s 605ms/step - loss: 1.8670 - accuracy: 0.2822 - val_loss: 1.6700 - val_accuracy: 0.3862
Epoch 2/50
384/384 [==============================] - 136s 354ms/step - loss: 1.6570 - accuracy: 0.3889 - val_loss: 1.6602 - val_accuracy: 0.3862
Epoch 3/50
384/384 [==============================] - 131s 342ms/step - loss: 1.6482 - accuracy: 0.3889 - val_loss: 1.6316 - val_accuracy: 0.3862
Epoch 4/50
384/384 [==============================] - 132s 345ms/step - loss: 1.6430 - accuracy: 0.3889 - val_loss: 1.6316 - val_accuracy: 0.3862
Epoch 5/50
384/384 [==============================] - 131s 340ms/step - loss: 1.6413 - accuracy: 0.3889 - val_loss: 1.6312 - val_accuracy: 0.3862
Epoch 6/50
384/384 [==============================] - 130s 339ms/step - loss: 1.6408 - accuracy: 0.3889 - val_loss: 1.6312 - val_accuracy: 0.3862
Epoch 7/50
384/384 [==============================] - 132s 343ms/step - loss: 1.6413 - accuracy: 0.3889 - val_loss: 1.6316 -